# Import library

In [1]:
import os

In [2]:
jar_packages = [
    "io.delta:delta-spark_2.12:3.3.0",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    # "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5",
    # "com.amazon.deequ:deequ:2.0.9-spark-3.5"
]

In [3]:
os.environ['HADOOP_USER_NAME'] = 'hadoopuser'
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages {",".join(jar_packages)} pyspark-shell'

In [4]:
import pyspark
import glob

from delta import *

from delta.tables import DeltaTable
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pprint import pprint

# Constants

In [5]:
MINIO_SERVER = "http://localhost:29000"
MINIO_ACCESS_KEY = "lLZ6r6vDyQvxq5WaTJ9w"
MINIO_SECRET_KEY = "WkxauVOq61tg0DnFQNSNnI8TbKfl9qxHBK3zILZ8"

# Spark Session

In [6]:
builder = (
    pyspark
    .sql.SparkSession.builder
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_SERVER)
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # Logging
    # .config("spark.log.level", "DEBUG")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
sc = spark.sparkContext # spark context

:: loading settings :: url = jar:file:/home/ad/Desktop/code/big-data-tools/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ad/.ivy2/cache
The jars for the packages stored in: /home/ad/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5156f373-5038-4686-bca5-96681fa7fc52;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 82ms :: artifacts dl 3ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hadoop#hadoop-

# CDC

## raw-sale-order-cdc

In [7]:
RAW_SALE_ORDER_CDC_PATH = "s3a://incremental-etl/raw-sale-order-cdc/"
raw_sale_order_cdc_df = (
    spark
    .read.format("delta")
    .load(RAW_SALE_ORDER_CDC_PATH)
)

raw_sale_order_cdc_df.show(truncate=False)

25/05/30 15:36:49 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/05/30 15:36:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+------------------+----------------+----------------+-----------------+-----------------+-----------------+-------------------------+--------------------------+-------------------------+----------------------+-------------------+------------------+--------------+--------------+--------------------------+-----------------+----------------+-------------------+-----------+------------+-----------------------+-------------+----------------+----------------+---------------------+--------------------+-----------+--------------------+---------------------+-----------------+-------------------+------------------------+-------------+------------------------+----------------------+------------------+----------------------+-----------------+----------------+-----------------+-------------------------+--------------------------------+-----------------------------+---------------+-------------------+---------------------------+------------------------+---------------------+--------------

In [8]:
# Incremental data from 1748415600000 to 1748419200000
(
    raw_sale_order_cdc_df
    # .filter(col("op") == "u")
    .filter(col("ts_ms") >= lit(1748415600000))
    # .filter(col("ts_ms") < lit(1748419200000))
    .show()
)

+---------+------------------+----------------+----------------+-----------------+-----------------+-----------------+-------------------------+--------------------------+-------------------------+----------------------+-------------------+------------------+--------------+--------------+--------------------------+-----------------+----------------+--------------------+-----------+------------+-----------------------+-------------+----------------+----------------+---------------------+--------------------+-----------+--------------------+---------------------+-----------------+-------------------+------------------------+-------------+------------------------+----------------------+------------------+----------------------+-----------------+----------------+-----------------+-------------------------+--------------------------------+-----------------------------+---------------+-------------------+---------------------------+------------------------+---------------------+-------------

## raw-res-partner-cdc

In [9]:
RAW_RES_PARTNER_CDC_PATH = "s3a://incremental-etl/raw-res-partner-cdc/"
raw_res_partner_cdc_df = (
    spark
    .read.format("delta")
    .load(RAW_RES_PARTNER_CDC_PATH)
)

raw_res_partner_cdc_df.show(truncate=False)

+---------+-----------------+------------------+-----------+------------+----------------+--------------+---------------+-----------------+------------------+------------+----------------------------+-----------------+----------------+--------------------+----------+-----------+---------+----------+-----------------------+--------------+---------------+-----------+-------------+--------------+----------+-----------+------------+------------+-------------+------------------------------+-------------------+-----------+--------------+-----------------------+------------------------+-------------+---------------+-----------------+--------------------+-----------------+---------------------+-----------------------+------------------+------------------------+-------------------+--------------+------------------+----------------------+----------------------+--------------------+--------------------+-------------------+-----------------------+------------------+--------------------------------

In [10]:
(
    raw_res_partner_cdc_df
    .filter(col("op") == 'u')
    .show()
)

+---------+-----------------+------------------+-----------+------------+----------------+--------------+---------------+-----------------+------------------+------------+----------------------------+-----------------+----------------+--------------------+----------+-----------+---------+----------+-----------------------+--------------------+---------------+-----------+-----------------+--------------+----------+-----------+--------------------+--------------+-------------+------------------------------+-------------------+-----------+--------------+-----------------------+------------------------+-------------+---------------+-----------------+--------------------+-----------------+---------------------+-----------------------+------------------+------------------------+--------------------+--------------+------------------+----------------------+----------------------+--------------------+--------------------+-------------------+-----------------------+------------------+-----------

## raw-sale-order-line-cdc

In [11]:
RAW_SALE_ORDER_LINE_CDC_PATH = "s3a://incremental-etl/raw-sale-order-line-cdc/"
raw_sale_order_line_cdc_df = (
    spark
    .read.format("delta")
    .load(RAW_SALE_ORDER_LINE_CDC_PATH)
)

raw_sale_order_line_cdc_df.show(truncate=False)

+---------+---------------+---------------+-----------------+------------------+-----------------------+------------------+-----------------+------------------+---------------------------+-----------------+----------------+------------+-------------------+---------------------------+---------------------+----------------------------+-----------+----------------------+-----------------+---------------+---------------------+------------------+---------------------------+--------------------------+--------------------+-------------------+---------------------+------------------------------+--------------------------------+---------------------+-----------------+------------------+-----------------+----------------+----------------------------+--------------------+---------------+--------+--------------+--------------+----------------+-----------------+----------------------+-----------------+----------------+-----------------+--------------------------+----------------+---------------+---

In [12]:
(
    raw_sale_order_line_cdc_df
    .filter(
        col("`after.id`") == lit(6)
    )
    .sort(desc("ts_ms"))
    .show(truncate=False)
)

+---------+---------------+---------------+-----------------+------------------+-----------------------+------------------+-----------------+------------------+---------------------------+-----------------+----------------+------------+-------------------+---------------------------+---------------------+----------------------------+-----------------------+----------------------+-----------------+---------------+---------------------+------------------+---------------------------+--------------------------+--------------------+-------------------+---------------------+------------------------------+--------------------------------+---------------------+-----------------+------------------+-----------------+----------------+----------------------------+--------------------+---------------+--------+--------------+--------------+----------------+-----------------+----------------------+-----------------+----------------+-----------------+--------------------------+----------------+-------

# Raw table

## raw_sale_order_line

In [13]:
RAW_SALE_ORDER_LINE_PATH = "s3a://incremental-etl/raw_sale_order_line/"
raw_sale_order_line_df = (
    spark
    .read.format("delta")
    .load(RAW_SALE_ORDER_LINE_PATH)
)

raw_sale_order_line_df.show(truncate=False)

+---+--------+----------+----------+-----------------------------------------------------------------------------------------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+
|id |order_id|partner_id|product_id|name                                                                                                 |product_uom_qty|price_unit|create_date        |write_date         |batch_id|lsn     |event_ts_ms  |event_ts           |delete_ts_ms|delete_ts|
+---+--------+----------+----------+-----------------------------------------------------------------------------------------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+
|1  |1       |10        |31        |[FURN_6666] Acoustic Bloc Screens                                                                    |3.0            |295

In [14]:
# expected: status=sent, product_uom_qty=15
(
    raw_sale_order_line_df
    .filter("id == 6")
    .show()
)

+---+--------+----------+----------+--------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+
| id|order_id|partner_id|product_id|                name|product_uom_qty|price_unit|        create_date|         write_date|batch_id|     lsn|  event_ts_ms|           event_ts|delete_ts_ms|delete_ts|
+---+--------+----------+----------+--------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+
|  6|       3|        12|         3|Virtual Interior ...|           15.0|     30.75|2025-05-24 16:54:20|2025-05-28 16:06:27|       2|83183088|1748423187049|2025-05-28 16:06:27|        NULL|     NULL|
+---+--------+----------+----------+--------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+


In [15]:
# change data feed
raw_sale_order_line_cdf = (
    spark.read.format("delta")
    .option("readChangeFeed", "true")
    # .option("startingTimestamp", '2021-04-21 05:45:46')
    .option("startingVersion", 0)
    .load(RAW_SALE_ORDER_LINE_PATH)
)

raw_sale_order_line_cdf.show(truncate=False)


+---+--------+----------+----------+-----------------------------------------------------------------------------------------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+----------------+---------------+-----------------------+
|id |order_id|partner_id|product_id|name                                                                                                 |product_uom_qty|price_unit|create_date        |write_date         |batch_id|lsn     |event_ts_ms  |event_ts           |delete_ts_ms|delete_ts|_change_type    |_commit_version|_commit_timestamp      |
+---+--------+----------+----------+-----------------------------------------------------------------------------------------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+----------------+---------------+-----------

In [16]:
# de-duplicate
(
    raw_sale_order_line_cdf
    .groupBy("id")
    .agg(
        max_by(struct("*"), "_commit_timestamp").alias("row")
    )
    .select("row.*")
    .show(truncate=False)
)

+---+--------+----------+----------+-----------------------------------------------------------------------------------------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+----------------+---------------+-----------------------+
|id |order_id|partner_id|product_id|name                                                                                                 |product_uom_qty|price_unit|create_date        |write_date         |batch_id|lsn     |event_ts_ms  |event_ts           |delete_ts_ms|delete_ts|_change_type    |_commit_version|_commit_timestamp      |
+---+--------+----------+----------+-----------------------------------------------------------------------------------------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+----------------+---------------+-----------

In [17]:
# review hash value
(
    raw_sale_order_line_cdf
    .filter("id == 6")
    .withColumn(
        "record_track_hash", hash(
            "product_id", "name", "product_uom_qty", "price_unit"
        )
    )
    .show()
)

+---+--------+----------+----------+--------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+----------------+---------------+--------------------+-----------------+
| id|order_id|partner_id|product_id|                name|product_uom_qty|price_unit|        create_date|         write_date|batch_id|     lsn|  event_ts_ms|           event_ts|delete_ts_ms|delete_ts|    _change_type|_commit_version|   _commit_timestamp|record_track_hash|
+---+--------+----------+----------+--------------------+---------------+----------+-------------------+-------------------+--------+--------+-------------+-------------------+------------+---------+----------------+---------------+--------------------+-----------------+
|  6|       3|        12|         3|Virtual Interior ...|           10.0|     30.75|2025-05-24 16:54:20|2025-05-24 16:54:20|       0|82749392|1748319491450|2025-05-27 11:18:11|        

# Data Modelling

## fact_order_line

In [50]:
FACT_ORDER_LINE_PATH = "s3a://incremental-etl/fact_order_line/"
fact_order_line_df = (
    spark
    .read.format("delta")
    .load(FACT_ORDER_LINE_PATH)
)

fact_order_line_df.show(truncate=False)

+-------+-------+--------+----------+---------------+-----------------------------------------------------------------------------------------------------+--------------------+---------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|line_sk|line_id|order_sk|partner_sk|line_product_id|line_name                                                                                            |line_product_uom_qty|line_price_unit|record_active_flag|record_start_date  |record_end_date    |record_track_hash|record_commit_timestamp|
+-------+-------+--------+----------+---------------+-----------------------------------------------------------------------------------------------------+--------------------+---------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|1      |1      |1       |10        |31             |[FURN_6666] Acoustic Bloc Screens                                

In [51]:
(
    fact_order_line_df
    .filter("line_id == 6")
    .show(truncate=False)
)

+-------+-------+--------+----------+---------------+----------------------------------------------------------------------------------------------------+--------------------+---------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|line_sk|line_id|order_sk|partner_sk|line_product_id|line_name                                                                                           |line_product_uom_qty|line_price_unit|record_active_flag|record_start_date  |record_end_date    |record_track_hash|record_commit_timestamp|
+-------+-------+--------+----------+---------------+----------------------------------------------------------------------------------------------------+--------------------+---------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|53     |6      |25      |12        |3              |Virtual Interior Design                                             

## dim_order

In [42]:
DIM_ORDER_PATH = "s3a://incremental-etl/dim_order/"
dim_order_df = (
    spark
    .read.format("delta")
    .load(DIM_ORDER_PATH)
)

dim_order_df.show(truncate=False)

+--------+--------+-------------------------+-------------------------+-------------------+--------------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|order_sk|order_id|order_name               |order_state              |order_date         |order_effective_date|record_active_flag|record_start_date  |record_end_date    |record_track_hash|record_commit_timestamp|
+--------+--------+-------------------------+-------------------------+-------------------+--------------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|26      |3       |S00003                   |sent                     |2025-05-24 16:54:21|NULL                |true              |2025-05-28 15:27:56|9999-12-31 07:00:00|186613446        |2025-05-28 16:00:17.111|
|1       |1       |S00001                   |draft                    |2025-04-24 16:54:00|NULL                |true              |2025-05-27 11

In [43]:
(
    dim_order_df
    .filter("order_id = 3")
    .show(truncate=False)
)

+--------+--------+-------------------------+-------------------------+-------------------+--------------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|order_sk|order_id|order_name               |order_state              |order_date         |order_effective_date|record_active_flag|record_start_date  |record_end_date    |record_track_hash|record_commit_timestamp|
+--------+--------+-------------------------+-------------------------+-------------------+--------------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|26      |3       |S00003                   |sent                     |2025-05-24 16:54:21|NULL                |true              |2025-05-28 15:27:56|9999-12-31 07:00:00|186613446        |2025-05-28 16:00:17.111|
|3       |3       |S00003                   |draft                    |2025-05-24 16:54:21|NULL                |false             |2025-05-27 11

In [ ]:
# build incremental query for dim_order
before_cdf = """
    select *
    from raw_sale_order
    where event_ts <= now()
    and event_ts >= now() - interval '3 days'
"""

after_cdf = """
    select *
    from table_changes('raw_sale_order', first_commit_ts, last_commit_ts) # only get changed records
    where event_ts <= now()
    and event_ts >= now() - interval '3 days'
"""

## dim_partner

In [40]:
DIM_PARTNER_PATH = "s3a://incremental-etl/dim_partner/"
dim_partner_df = (
    spark
    .read.format("delta")
    .load(DIM_PARTNER_PATH)
)

dim_partner_df.show(truncate=False)

+----------+----------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|partner_sk|partner_id|partner_name                                      |partner_email                                     |partner_phone                                     |partner_street                                    |partner_city                                      |record_active_flag|record_start_date  |record_end_date    |record_track_hash|record_commit_timestamp|
+----------+----------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+-------------

In [41]:
(
    dim_partner_df
    .filter("partner_id = 12")
    .show(truncate=False)
)

+----------+----------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|partner_sk|partner_id|partner_name                                      |partner_email                                     |partner_phone                                     |partner_street                                    |partner_city                                      |record_active_flag|record_start_date  |record_end_date    |record_track_hash|record_commit_timestamp|
+----------+----------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+-------------

# Point-in-time table

In [44]:
FACT_ORDER_LINE_PATH = "s3a://incremental-etl/fact_order_line/"
fact_order_line_df = (
    spark
    .read.format("delta")
    .load(FACT_ORDER_LINE_PATH)
)

fact_order_line_df.show(truncate=False)

+-------+-------+--------+----------+---------------+-----------------------------------------------------------------------------------------------------+--------------------+---------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|line_sk|line_id|order_sk|partner_sk|line_product_id|line_name                                                                                            |line_product_uom_qty|line_price_unit|record_active_flag|record_start_date  |record_end_date    |record_track_hash|record_commit_timestamp|
+-------+-------+--------+----------+---------------+-----------------------------------------------------------------------------------------------------+--------------------+---------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|1      |1      |1       |10        |31             |[FURN_6666] Acoustic Bloc Screens                                

In [45]:
DIM_ORDER_PATH = "s3a://incremental-etl/dim_order/"
dim_order_df = (
    spark
    .read.format("delta")
    .load(DIM_ORDER_PATH)
)

dim_order_df.show(truncate=False)

+--------+--------+-------------------------+-------------------------+-------------------+--------------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|order_sk|order_id|order_name               |order_state              |order_date         |order_effective_date|record_active_flag|record_start_date  |record_end_date    |record_track_hash|record_commit_timestamp|
+--------+--------+-------------------------+-------------------------+-------------------+--------------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|26      |3       |S00003                   |sent                     |2025-05-24 16:54:21|NULL                |true              |2025-05-28 15:27:56|9999-12-31 07:00:00|186613446        |2025-05-28 16:00:17.111|
|1       |1       |S00001                   |draft                    |2025-04-24 16:54:00|NULL                |true              |2025-05-27 11

In [46]:
# point-in-time join
(
    fact_order_line_df
    .join(
        dim_order_df, "order_sk"
    )
    .select(
        fact_order_line_df.line_sk,
        fact_order_line_df.line_id,
        fact_order_line_df.order_sk,
        dim_order_df.order_id,
        dim_order_df.record_active_flag,
        dim_order_df.record_start_date,
        dim_order_df.record_end_date
    )
    .join(
        dim_order_df.filter(dim_order_df.record_active_flag == True),
        "order_id"
    )
    .show(truncate=False)
)

+--------+-------+-------+--------+------------------+-------------------+-------------------+--------+-------------------------+-------------------------+-------------------+--------------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|order_id|line_sk|line_id|order_sk|record_active_flag|record_start_date  |record_end_date    |order_sk|order_name               |order_state              |order_date         |order_effective_date|record_active_flag|record_start_date  |record_end_date    |record_track_hash|record_commit_timestamp|
+--------+-------+-------+--------+------------------+-------------------+-------------------+--------+-------------------------+-------------------------+-------------------+--------------------+------------------+-------------------+-------------------+-----------------+-----------------------+
|1       |1      |1      |1       |true              |2025-05-27 11:18:11|9999-12-31 07:00:00|1       |S00

In [49]:
spark.sql(f"""
    select
        f.line_sk,
        f.line_id,
        f.order_sk,
        do.order_id as fact_order_id,
        do.order_state,
        do_latest.order_sk as dim_order_sk,
        do_latest.order_state
    from delta.`{FACT_ORDER_LINE_PATH}` f
    join delta.`{DIM_ORDER_PATH}` do on f.order_sk = do.order_sk
    join delta.`{DIM_ORDER_PATH}` do_latest on do.order_id = do_latest.order_id
    where
        f.record_active_flag = true
        and do_latest.record_active_flag = true
        and f.line_id in (6, 7)
""").show(truncate=False)


+-------+-------+--------+-------------+-------------------------+------------+-------------------------+
|line_sk|line_id|order_sk|fact_order_id|order_state              |dim_order_sk|order_state              |
+-------+-------+--------+-------------+-------------------------+------------+-------------------------+
|7      |7      |3       |3            |draft                    |26          |sent                     |
+-------+-------+--------+-------------+-------------------------+------------+-------------------------+



# Aggregation

## Weekly Sales

In [52]:
# weekly_sales: date_order, product_id, sales_qty (order_state = sale)

In [73]:
weekly_sales_agg_sql = f"""
    with product_sales as (
        select
            do.order_id,
            f.line_id,
            f.line_product_id,
            f.line_product_uom_qty,
            do_latest.order_state,
            do_latest.order_date,
            date_trunc('week', do_latest.order_date) as week_date
        from delta.`{FACT_ORDER_LINE_PATH}` f
        join delta.`{DIM_ORDER_PATH}` do
            on f.order_sk = do.order_sk
        join delta.`{DIM_ORDER_PATH}` do_latest
            on do_latest.order_id = do.order_id
        where
            do_latest.record_active_flag = true
            and f.record_active_flag = true
    )
        select
            line_product_id,
            week_date,
            order_state,
            sum(line_product_uom_qty) as sold_qty
        from product_sales
        group by line_product_id, week_date, order_state
        order by 1, 2
"""
spark.sql(weekly_sales_agg_sql).show(truncate=False)

+---------------+-------------------+-------------------------+--------+
|line_product_id|week_date          |order_state              |sold_qty|
+---------------+-------------------+-------------------------+--------+
|3              |2025-04-21 00:00:00|sale                     |24.0    |
|3              |2025-04-21 00:00:00|draft                    |24.0    |
|3              |2025-05-19 00:00:00|sale                     |16.0    |
|4              |2025-04-21 00:00:00|draft                    |30.0    |
|4              |2025-04-21 00:00:00|sale                     |30.0    |
|5              |2025-04-21 00:00:00|draft                    |2.0     |
|5              |2025-04-21 00:00:00|sent                     |2.0     |
|5              |2025-05-19 00:00:00|sent                     |1.0     |
|5              |2025-05-19 00:00:00|sale                     |3.0     |
|6              |2025-04-21 00:00:00|sent                     |5.0     |
|6              |2025-04-21 00:00:00|draft         

In [ ]:
# incremental aggregation
# 1. get changed data and identify the timestamp we re-calculate (week_start, week_end)
# 2. re-run aggregation for affected timestamps
# 3. merge changes into the current aggregation table e.g MERGE INTO

In [7]:
AGG_DAILY_PRODUCT_SALE_DELTA_PATH = 's3a://incremental-etl/daily_product_sales'

daily_product_sales_df = (
    spark
    .read.format("delta")
    .load(AGG_DAILY_PRODUCT_SALE_DELTA_PATH)
    .sort("product_id", "date_sk")
)
daily_product_sales_df.show(n=50, truncate=False)

25/06/03 15:48:51 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/06/03 15:48:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+----------+----------+--------+-----------------+-----------------------+
|daily_sales_sk|date_sk   |product_id|sold_qty|record_track_hash|record_commit_timestamp|
+--------------+----------+----------+--------+-----------------+-----------------------+
|8             |2025-04-24|3         |96.0    |900622957        |2025-05-28 18:10:00.66 |
|1             |2025-05-24|3         |176.0   |1134485390       |2025-05-28 18:10:00.66 |
|21            |2025-04-24|4         |120.0   |317437054        |2025-05-28 18:10:00.66 |
|17            |2025-04-24|5         |8.0     |485499514        |2025-05-28 18:10:00.66 |
|12            |2025-05-24|5         |12.0    |2001318435       |2025-05-28 18:10:00.66 |
|3             |2025-04-24|6         |33.0    |-1529143666      |2025-05-28 18:10:00.66 |
|19            |2025-05-24|6         |22.0    |55869459         |2025-05-28 18:10:00.66 |
|7             |2025-05-24|12        |1.0     |2054339909       |2025-05-28 18:10:00.66 |
|2        

## Customer Revenue

In [ ]:
# customer_revenue: date_order, customer_id, revenue (product_qty * price_unit)